# MT5 Data Ingestion — Save to CSV

This notebook only handles data ingestion from MetaTrader 5 (MT5) and saves the result to CSV.
- Works on Windows with MT5 64-bit installed and logged-in.
- Uses robust chunked fetching and ensures the symbol is visible in Market Watch.
- Comments are in English for clarity.

In [1]:
# Install required packages for ingestion only
!pip install -q numpy pandas pytz tqdm MetaTrader5



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


## Parameters
Adjust these before running. If you have multiple MT5 terminals, set `TERMINAL_PATH`.
If MT5 is already open and logged-in, you can leave `TERMINAL_PATH = None`.

In [2]:
import os, struct, platform
from datetime import datetime, timedelta

# MT5 terminal path (use raw string or forward slashes). Set to None to rely on a running instance.
TERMINAL_PATH = None  # e.g. r'C:\Program Files\MetaTrader 5 EXNESS\terminal64.exe'

# Symbol and timeframe (ensure the exact broker symbol, e.g., 'XAUUSD', 'XAUUSDm', etc.)
SYMBOL = 'XAUUSDm'
TIMEFRAME = 'M1'        # One of: M1/M5/M15/M30/H1/H4/D1...
YEARS_BACK = 3          # How many years back to fetch

# Output directory and filename (auto timestamp if None)
OUTPUT_DIR = 'data'
OUTPUT_FILENAME = TIMEFRAME + "_Y"+ str(YEARS_BACK) + '.csv'  # e.g., 'XAUUSD_M5.csv' or leave None to auto-name

print('Python arch:', struct.calcsize('P')*8, 'bit')
print('Windows version:', platform.version())


Python arch: 64 bit
Windows version: 10.0.26100


## Connect to MT5
- Tries to initialize with explicit path first, then without path (requires MT5 running).
- Print diagnostics and last error if initialization fails.

In [3]:
import MetaTrader5 as mt5
import contextlib
# Clean any previous session
with contextlib.suppress(Exception):
    mt5.shutdown()

def try_init(with_path: bool = True) -> bool:
    """Try to initialize MT5 with or without explicit terminal path.
    Add login/server/password here if your terminal is not pre-logged-in.
    """
    if with_path and TERMINAL_PATH:
        ok = mt5.initialize(path=TERMINAL_PATH)
    elif with_path:
        # No explicit path provided
        ok = mt5.initialize()
    else:
        ok = mt5.initialize()
    if not ok:
        print('init failed:', mt5.last_error())
    return ok

# 1) Try with provided path (if any)
if not try_init(with_path=True):
    # 2) Try without path (requires MT5 running and logged-in)
    print('Retrying without explicit path... (make sure MT5 is running)')
    if not try_init(with_path=False):
        raise RuntimeError(f'Could not initialize MT5. Last error: {mt5.last_error()}')

print('MT5 connected. Version:', mt5.version())
print('Account:', mt5.account_info())
print('Terminal:', mt5.terminal_info())


MT5 connected. Version: (500, 5331, '8 Oct 2025')
Account: AccountInfo(login=206414252, trade_mode=0, leverage=2000, limit_orders=1024, margin_so_mode=0, trade_allowed=False, trade_expert=True, margin_mode=2, currency_digits=2, fifo_close=False, balance=0.0, credit=0.0, profit=0.0, equity=0.0, margin=0.0, margin_free=0.0, margin_level=0.0, margin_so_call=60.0, margin_so_so=0.0, margin_initial=0.0, margin_maintenance=0.0, assets=0.0, liabilities=0.0, commission_blocked=0.0, name='Standard', server='Exness-MT5Trial7', currency='USD', company='Exness Technologies Ltd')
Terminal: TerminalInfo(community_account=False, community_connection=False, connected=True, dlls_allowed=False, trade_allowed=False, tradeapi_disabled=False, email_enabled=False, ftp_enabled=False, notifications_enabled=False, mqid=False, build=5331, maxbars=100000, codepage=0, ping_last=27022, community_balance=0.0, retransmission=0.0, company='Exness Technologies Ltd', name='MetaTrader 5 EXNESS', language='English', path=

## Symbol discovery (optional)
List symbols that contain 'XAU' to help you pick the exact broker symbol.

In [4]:
# Discover available XAU symbols
cands = mt5.symbols_get('*XAU*')
print('Found XAU-like symbols:', [s.name for s in cands])
# Make sure the target symbol is visible in Market Watch
mt5.symbol_select(SYMBOL, True)


Found XAU-like symbols: ['BTCXAUm', 'XAUAUDm', 'XAUEURm', 'XAUGBPm', 'XAUUSDm']


True

## Robust fetch utilities
Chunked copy using `copy_rates_range` in 30-day windows. Ensures symbol visibility and returns a tidy DataFrame.

In [5]:
import pandas as pd
import numpy as np
import contextlib
from tqdm.auto import tqdm

# Map string timeframe to MT5 constant
def _infer_mt5_timeframe(tf: str):
    mapping = {
        'M1': mt5.TIMEFRAME_M1, 'M2': mt5.TIMEFRAME_M2, 'M3': mt5.TIMEFRAME_M3, 'M4': mt5.TIMEFRAME_M4,
        'M5': mt5.TIMEFRAME_M5, 'M6': mt5.TIMEFRAME_M6, 'M10': mt5.TIMEFRAME_M10, 'M12': mt5.TIMEFRAME_M12,
        'M15': mt5.TIMEFRAME_M15, 'M20': mt5.TIMEFRAME_M20, 'M30': mt5.TIMEFRAME_M30,
        'H1': mt5.TIMEFRAME_H1, 'H2': mt5.TIMEFRAME_H2, 'H3': mt5.TIMEFRAME_H3, 'H4': mt5.TIMEFRAME_H4,
        'H6': mt5.TIMEFRAME_H6, 'H8': mt5.TIMEFRAME_H8, 'H12': mt5.TIMEFRAME_H12,
        'D1': mt5.TIMEFRAME_D1, 'W1': mt5.TIMEFRAME_W1, 'MN1': mt5.TIMEFRAME_MN1
    }
    return mapping[tf]

def ensure_symbol_visible(symbol: str):
    info = mt5.symbol_info(symbol)
    if info is None:
        raise ValueError(f'Symbol not found on this server: {symbol}')
    if not info.visible:
        if not mt5.symbol_select(symbol, True):
            raise RuntimeError(f'symbol_select failed for {symbol}')

def mt5_fetch_rates_robust(symbol: str, timeframe: str, years_back: int) -> pd.DataFrame:
    """
    Fetch OHLCV data from MT5 in 30-day chunks to improve reliability.
    Uses UTC-naive datetimes per MT5 API expectations. Returns a tidy DataFrame indexed by UTC.
    """
    tf = _infer_mt5_timeframe(timeframe)
    ensure_symbol_visible(symbol)

    date_to = datetime.utcnow()
    date_from = date_to - timedelta(days=365*years_back)

    frames = []
    step = timedelta(days=30)
    cur = date_from
    with tqdm(total=(date_to - date_from).days//30 + 1, desc='Fetching') as pbar:
        while cur < date_to:
            nxt = min(cur + step, date_to)
            part = mt5.copy_rates_range(symbol, tf, cur, nxt)
            if part is None:
                print('copy_rates_range failed:', mt5.last_error(), cur, nxt)
                cur = nxt
                pbar.update(1)
                continue
            if len(part) == 0:
                cur = nxt
                pbar.update(1)
                continue
            frames.append(pd.DataFrame(part))
            cur = nxt
            pbar.update(1)

    if not frames:
        raise ValueError('No data returned — check symbol name, Market Watch visibility, login, and timeframe.')

    df = pd.concat(frames, ignore_index=True)
    df.drop_duplicates(subset='time', inplace=True)
    df['time'] = pd.to_datetime(df['time'], unit='s', utc=True)
    df = df.rename(columns={'open':'Open','high':'High','low':'Low','close':'Close',
                            'tick_volume':'TickVolume','spread':'Spread'})
    cols = ['time','Open','High','Low','Close','TickVolume','Spread','real_volume']
    df = df[cols].set_index('time').sort_index()
    return df


c:\Users\sukum\OneDrive\Desktop\auto_trading_project\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Run fetch and save CSV
Creates the output directory if needed and writes a CSV with a timestamp by default.

In [6]:
# Fetch data
df = mt5_fetch_rates_robust(SYMBOL, TIMEFRAME, YEARS_BACK)
print('Data shape:', df.shape)
display(df.head())

# Ensure output directory exists
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Build filename if not provided
if OUTPUT_FILENAME is None:
    ts = datetime.utcnow().strftime('%Y%m%d_%H%M%S')
    OUTPUT_FILENAME = f'{SYMBOL}_{TIMEFRAME}_{ts}.csv'

out_path = os.path.join(OUTPUT_DIR, OUTPUT_FILENAME)
df.to_csv(out_path, index=True)
print('Saved CSV to:', out_path)


C:\Users\sukum\AppData\Local\Temp\ipykernel_22512\466593058.py:34: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  date_to = datetime.utcnow()
Fetching: 100%|██████████| 37/37 [00:00<00:00, 2705.81it/s]

Data shape: (99655, 7)


,Open,High,Low,Close,TickVolume,Spread,real_volume
time,,,,,,,
2025-07-03 18:02:00+00:00,3328.741,3329.104,3328.741,3329.049,46,160,0
2025-07-03 18:03:00+00:00,3329.049,3329.049,3328.882,3329.027,23,160,0
2025-07-03 18:04:00+00:00,3329.026,3329.026,3328.598,3328.737,57,160,0
2025-07-03 18:05:00+00:00,3328.740,3329.134,3328.647,3329.016,52,160,0
2025-07-03 18:06:00+00:00,3329.015,3329.129,3328.962,3328.962,26,160,0


Saved CSV to: data\M1_Y3.csv


## Cleanup (optional)
Shutdown the MT5 connection when done.

In [7]:
with contextlib.suppress(Exception):
    mt5.shutdown()
print('MT5 shutdown.')

MT5 shutdown.
